# CMB Header Image Generator

This notebook generates a Cosmic Microwave Background (CMB) visualization using CAMB and HealPy.
The output will be used as the header background for the website.

In [ ]:
# Import required libraries
import numpy as np
import healpy as hp
import camb
import matplotlib.pyplot as plt
import os

ModuleNotFoundError: No module named 'numpy'

## Set Cosmological Parameters (Planck 2018)

Using CAMB to set up the standard Planck 2018 cosmological parameters.

In [ ]:
# Set up CAMB parameters (Planck 2018 cosmology)
pars = camb.CAMBparams()
pars.set_cosmology(
    H0=67.4,  # Hubble constant
    ombh2=0.0224,  # Baryon density
    omch2=0.120,  # Cold dark matter density
    tau=0.054,  # Optical depth to reionization
    mnu=0.06,  # Sum of neutrino masses
    omk=0,  # Curvature (flat universe)
)
pars.InitPower.set_params(As=2.1e-9, ns=0.965)  # Scalar amplitude  # Spectral index

# Set maximum multipole for power spectrum
NSIDE = 2048
LMAX = 3 * NSIDE - 1
pars.set_for_lmax(LMAX)

print(f"NSIDE: {NSIDE}")
print(f"LMAX: {LMAX}")
print(f"Number of pixels: {12 * NSIDE**2:,}")

## Generate CMB Power Spectrum

Compute the theoretical CMB power spectrum using CAMB.

In [ ]:
# Get the CMB power spectrum from CAMB
results = camb.get_results(pars)
powers = results.get_cmb_power_spectra(pars, CMB_unit="muK")

# Extract the TT (temperature-temperature) spectrum
# The output is in D_l = l(l+1)C_l/(2*pi) format
cl_tt = powers["total"][:, 0]

# Convert from D_l to C_l for HealPy
ells = np.arange(len(cl_tt))
cl_for_hp = np.zeros(LMAX + 1)

# Avoid division by zero at l=0,1
for l in range(2, min(len(cl_tt), LMAX + 1)):
    cl_for_hp[l] = cl_tt[l] * 2 * np.pi / (l * (l + 1))

print(f"Power spectrum computed up to l={len(cl_tt)-1}")

## Generate HealPix CMB Map

Create a random CMB realization from the power spectrum using HealPy's synfast function.

In [ ]:
# Generate a random CMB map realization
np.random.seed(42)  # For reproducibility
cmb_map = hp.synfast(cl_for_hp, nside=NSIDE, lmax=LMAX, new=True)

print(f"CMB map generated with shape: {cmb_map.shape}")
print(f"Temperature range: {cmb_map.min():.1f} to {cmb_map.max():.1f} μK")

## Create Header Image with Dark Background

Render the CMB map with a Mollweide projection on a pleasant dark background, suitable for use as a website header.

In [ ]:
# Pleasant dark background color (dark navy/slate)
DARK_BG = "#1a1a2e"

# Create output directory if it doesn't exist
output_dir = "../assets/imgs"
os.makedirs(output_dir, exist_ok=True)

# Create figure with dark background
fig = plt.figure(figsize=(20, 10), facecolor=DARK_BG)

# Plot the CMB map using Mollweide projection
hp.mollview(
    cmb_map,
    fig=fig.number,
    title="",  # No title for header image
    unit="",  # No unit label
    cmap="viridis",  # Blue-green colormap
    min=-300,  # Temperature range in μK
    max=300,
    cbar=False,  # No colorbar for clean header
    bgcolor=DARK_BG,  # Set background color
    hold=True,
)

# Set dark background for all axes
for ax in fig.axes:
    ax.set_facecolor(DARK_BG)
    ax.patch.set_facecolor(DARK_BG)

fig.patch.set_facecolor(DARK_BG)

# Save the image with higher resolution (300 dpi)
output_path = os.path.join(output_dir, "cmb_header.png")
plt.savefig(
    output_path,
    dpi=300,  # Higher resolution
    facecolor=DARK_BG,
    edgecolor=DARK_BG,
    bbox_inches="tight",
    pad_inches=0,
)

plt.show()
print(f"Header image saved to: {output_path}")

## DIS Feynman Diagram with Color Dipole

Create a Feynman diagram showing Deep Inelastic Scattering (DIS) in the dipole picture, where the virtual photon splits into a quark-antiquark dipole that interacts with the target.

In [ ]:
# Create DIS Feynman Diagram with Color Dipole Picture
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyArrowPatch, Arc, Circle, FancyBboxPatch
import numpy as np

# Use colors that match the viridis/teal theme
DARK_BG = "#1a1a2e"
TEAL = "#7ed3d8"
GREEN = "#4dc98b"
YELLOW = "#fde725"  # viridis yellow
PURPLE = "#440154"  # viridis purple

fig, ax = plt.subplots(figsize=(12, 8), facecolor=DARK_BG)
ax.set_facecolor(DARK_BG)
ax.set_xlim(-1, 11)
ax.set_ylim(-1, 9)
ax.set_aspect("equal")
ax.axis("off")


# Helper function for wavy lines (photon propagator)
def draw_wavy_line(ax, start, end, color, amplitude=0.15, periods=6, lw=2):
    x1, y1 = start
    x2, y2 = end
    length = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    angle = np.arctan2(y2 - y1, x2 - x1)
    t = np.linspace(0, 1, 200)
    wave = amplitude * np.sin(2 * np.pi * periods * t)
    x = x1 + t * (x2 - x1) - wave * np.sin(angle)
    y = y1 + t * (y2 - y1) + wave * np.cos(angle)
    ax.plot(x, y, color=color, lw=lw, solid_capstyle="round")


# Helper function for gluon (coiled/spring) lines
def draw_gluon_line(ax, start, end, color, amplitude=0.2, periods=8, lw=2):
    x1, y1 = start
    x2, y2 = end
    length = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    angle = np.arctan2(y2 - y1, x2 - x1)
    t = np.linspace(0, 1, 300)
    # Create coiled appearance
    wave_x = amplitude * np.sin(2 * np.pi * periods * t)
    wave_y = amplitude * 0.3 * np.cos(2 * np.pi * periods * t)
    x = x1 + t * (x2 - x1) - wave_x * np.sin(angle) + wave_y * np.cos(angle)
    y = y1 + t * (y2 - y1) + wave_x * np.cos(angle) + wave_y * np.sin(angle)
    ax.plot(x, y, color=color, lw=lw, solid_capstyle="round")


# Draw incoming electron (left side)
ax.annotate(
    "", xy=(2, 7), xytext=(0, 7), arrowprops=dict(arrowstyle="->", color=TEAL, lw=2)
)
ax.text(
    -0.3,
    7,
    r"$e^-$",
    fontsize=16,
    color=TEAL,
    ha="right",
    va="center",
    fontweight="bold",
)

# Draw outgoing electron (upper left)
ax.annotate(
    "", xy=(0.5, 8.5), xytext=(2, 7), arrowprops=dict(arrowstyle="->", color=TEAL, lw=2)
)
ax.text(
    0.2,
    8.7,
    r"$e^{-'}$",
    fontsize=16,
    color=TEAL,
    ha="right",
    va="center",
    fontweight="bold",
)

# Draw virtual photon (wavy line from electron vertex down to dipole)
draw_wavy_line(ax, (2, 7), (4, 5), YELLOW, amplitude=0.12, periods=5, lw=2.5)
ax.text(
    2.3,
    6.3,
    r"$\gamma^*$",
    fontsize=16,
    color=YELLOW,
    ha="left",
    va="center",
    fontweight="bold",
)

# Draw the quark-antiquark dipole (from photon split point)
# Quark line
ax.annotate(
    "", xy=(7, 6), xytext=(4, 5), arrowprops=dict(arrowstyle="->", color=GREEN, lw=2.5)
)
ax.text(
    5.2,
    6.0,
    r"$q$",
    fontsize=16,
    color=GREEN,
    ha="center",
    va="bottom",
    fontweight="bold",
)

# Antiquark line
ax.annotate(
    "",
    xy=(7, 4),
    xytext=(4, 5),
    arrowprops=dict(arrowstyle="->", color="#ff6b6b", lw=2.5),
)
ax.text(
    5.2,
    4.2,
    r"$\bar{q}$",
    fontsize=16,
    color="#ff6b6b",
    ha="center",
    va="top",
    fontweight="bold",
)

# Draw the dipole interaction with target (gluon exchanges)
draw_gluon_line(ax, (6, 5.5), (6, 2.5), "#ffa500", amplitude=0.18, periods=6, lw=2)
draw_gluon_line(ax, (6.8, 4.8), (6.8, 2.5), "#ffa500", amplitude=0.18, periods=5, lw=2)
ax.text(
    7.3,
    3.5,
    r"$g$",
    fontsize=14,
    color="#ffa500",
    ha="left",
    va="center",
    fontweight="bold",
)

# Draw the proton/target (blob at bottom)
target = FancyBboxPatch(
    (3, 0.5),
    6,
    2,
    boxstyle="round,pad=0.1,rounding_size=0.8",
    facecolor="#2d5a7b",
    edgecolor=TEAL,
    lw=2,
    alpha=0.8,
)
ax.add_patch(target)
ax.text(
    6,
    1.5,
    r"$p$ (target)",
    fontsize=16,
    color=TEAL,
    ha="center",
    va="center",
    fontweight="bold",
)

# Draw outgoing quark and antiquark (after interaction)
ax.annotate(
    "", xy=(10, 7), xytext=(7, 6), arrowprops=dict(arrowstyle="->", color=GREEN, lw=2.5)
)
ax.text(
    10.2, 7, r"$q$", fontsize=16, color=GREEN, ha="left", va="center", fontweight="bold"
)

ax.annotate(
    "",
    xy=(10, 3),
    xytext=(7, 4),
    arrowprops=dict(arrowstyle="->", color="#ff6b6b", lw=2.5),
)
ax.text(
    10.2,
    3,
    r"$\bar{q}$",
    fontsize=16,
    color="#ff6b6b",
    ha="left",
    va="center",
    fontweight="bold",
)

# Add dipole bracket/indicator
dipole_arc = Arc(
    (7, 5), 0.8, 2.5, angle=0, theta1=-90, theta2=90, color=TEAL, lw=1.5, linestyle="--"
)
ax.add_patch(dipole_arc)
ax.text(
    7.8, 5, "dipole", fontsize=12, color=TEAL, ha="left", va="center", style="italic"
)

# Title
ax.text(
    5,
    8.5,
    "Deep Inelastic Scattering: Dipole Picture",
    fontsize=18,
    color=TEAL,
    ha="center",
    va="center",
    fontweight="bold",
)

# Add momentum labels
ax.text(
    1, 7.3, r"$k$", fontsize=12, color=TEAL, ha="center", va="bottom", style="italic"
)
ax.text(
    3.5,
    5.8,
    r"$q$",
    fontsize=12,
    color=YELLOW,
    ha="center",
    va="bottom",
    style="italic",
)

plt.tight_layout()

# Save the figure
output_path = "../assets/imgs/dis_feynman.png"
plt.savefig(
    output_path,
    dpi=300,
    facecolor=DARK_BG,
    edgecolor=DARK_BG,
    bbox_inches="tight",
    pad_inches=0.2,
)
plt.show()
print(f"DIS Feynman diagram saved to: {output_path}")